In [17]:
class InitOnAccess:
    def __init__(self, kclass, *args, **kwargs):
        self.kclass = kclass
        self.args = args
        self.kwargs = kwargs
        self._initialized = None
    def __get__(self, instance, owner):
        if self._initialized is None:
            print("Initialized")
            self._initialized = self.kclass(*self.args, **self.kwargs)
        else:
            print("cached")
        return self._initialized

In [32]:
class MyClass:
    lazily_initialized = InitOnAccess(list, "argument")  # 不能放在init中

In [33]:
m = MyClass()

In [34]:
m.lazily_initialized

Initialized


['a', 'r', 'g', 'u', 'm', 'e', 'n', 't']

In [35]:
m.lazily_initialized

cached


['a', 'r', 'g', 'u', 'm', 'e', 'n', 't']

In [37]:
class lazy_property:  # 既是 descriptor 又是 decorator
    def __init__(self, function):
        self.fget = function
    def __get__(self, instance, owner):  # instance 是实例，owner是所有者的类
        value = self.fget(instance)  # 直接调用实例（ function(self))
        setattr(instance, self.fget.__name__, value)  # 增加属性，类似property
        return value

In [47]:
class MyClass:
    def __init__(self, radius):
        self.radius = radius
    @property  # 只起到了延迟加载作用
    def area(self):
        print("evaluate")
        return 3.14 * self.radius ** 2

In [51]:
m = MyClass(10)
for _ in range(5):
    m.area  # will cal 5 times

evaluate
evaluate
evaluate
evaluate
evaluate


In [52]:
m.__dict__  # 属性栏里并没有area

{'radius': 10}

In [53]:
class MyClass:
    def __init__(self, radius):
        self.radius = radius
    @lazy_property  # 只起到了延迟加载作用
    def area(self):
        print("evaluate")
        return 3.14 * self.radius ** 2

In [54]:
m = MyClass(10)
for _ in range(5):
    m.area  # just call once

evaluate


In [55]:
m.__dict__

{'radius': 10, 'area': 314.0}